# Setup

In [ ]:
 %run "Path to the setup.ipynb file"

importing standardlib modules
importing google drive
importing tensorflow/keras modules
importing miscelaneaous modules


# Functions for Sampling and Models

In [ ]:
# @title Sample data/target batches

def load_batch_conversation(conversation_df: DataFrame,
                            ec_pairs_df: DataFrame,
                            embeddings_path: str,
                            batch_size: int = 1,
                            validation: bool = False
                            ) -> Tuple[np.ndarray, np.ndarray, np.ndarray, list]:
  conversations_included = set([-1])
  conversations = list()
  emotions = list()
  causes = list()
  range_ = (data_params["start_validation"], data_params["start_testing"]) \
    if validation else (0, data_params["start_validation"])
  if batch_size > -np.subtract(*range_):
    print(f"Requested batch size exceeds available data, max available {len(range_)}")
    batch_size = len(range_)
  conv_ind = np.arange(*range_)
  rng.shuffle(conv_ind)

  for index_selected in tqdm(conv_ind[:batch_size]):
    # load embeddings for data
    try:
        embeddings = np.load(embeddings_path + f"/conv_{index_selected}_embeddings.npy")
    except OSError:
      raise OSError(f"Embedding file conv_{index_selected}_embeddings.npy not found or could not be read")
      return

    # load emotions
    emotions_raw = conversation_df.loc[conversation_df["conversation_ID"] == index_selected]["emotion"].values
    emotions_one_hot = [tf.one_hot(data_params["emotions"].index(emotion), data_params["n_emotions"])
        for emotion in emotions_raw]

    # load causes
    causes_conv = ec_pairs_df.loc[ec_pairs_df["conversation_ID"] == index_selected]
    existing_causes = causes_conv[["emotion_utterance", "cause_utterance"]]
    cause_matrix = np.zeros((data_params["len_conversation"], data_params["len_conversation"]))
    for _, row in existing_causes.iterrows():
      if row["emotion_utterance"] < data_params["len_conversation"] and \
         row["cause_utterance"] < data_params["len_conversation"]:
        cause_matrix[row["emotion_utterance"]-1, row["cause_utterance"]-1] = 1

    # pad conversations with empty utterances
    if data_params["len_conversation"] > len(embeddings):
      len_padding = data_params["len_conversation"] - len(embeddings)

      emb_padding = np.zeros((len_padding, 1, 40, 1024))
      embeddings = np.append(embeddings, emb_padding, axis=0)

      emo_padding = tf.one_hot(np.repeat(2, len_padding), data_params["n_emotions"])
      emotions_one_hot = np.append(emotions_one_hot, emo_padding, axis=0)


    conversations.append(embeddings[:data_params["len_conversation"]])
    emotions.append(emotions_one_hot[:data_params["len_conversation"]])
    causes.append(cause_matrix.flatten())

  return (np.array(conversations).squeeze(2), np.array(emotions),
    np.array(causes), conv_ind[:batch_size])

In [ ]:
# @title get Fully Connected NN

def get_fc_nn(num_utterances,
              num_tokens,
              embedding_dim,
              num_emotion_classes,
              output_dim,
              dropout,
              l1,
              l2,
              **kwargs
              ):

  utterance_embeddings_input = Input(shape=(num_utterances, embedding_dim), name="utterance_embeddings_input")
  emotion_vectors_input = Input(shape=(num_utterances, num_emotion_classes),
                                name="emotion_vectors_input")  # Emotion vector for each utteranch

  # Concatenate the pooled utterance embeddings along the feature dimension
  concatenated_embeddings = Concatenate(axis=2)([utterance_embeddings_input, emotion_vectors_input])
  # Concatenate emotion vectors with the concatenated embeddings
  # Define a feedforward network for predicting causes for each utterance
  dense_block = Dense(units=1024,
                      activation="relu",
                      kernel_regularizer=regularizers.L1L2(l1=l1, l2=l2)
                      )(concatenated_embeddings)
  dense_block = Dropout(dropout)(dense_block)
  dense_block = Dense(units=512,
                      activation="relu",
                      kernel_regularizer=regularizers.L1L2(l1=l1, l2=l2)
                      )(dense_block)
  dense_block = Dropout(dropout)(dense_block)
  dense_block = Dense(units=512,
                      activation="relu",
                      kernel_regularizer=regularizers.L1L2(l1=l1, l2=l2)
                      )(dense_block)
  dense_out = Flatten()(dense_block)

  cause_prediction = Dense(output_dim, activation="sigmoid")(dense_out)

  # Create the model
  model = Model(inputs=[utterance_embeddings_input, emotion_vectors_input], outputs=cause_prediction)
  return model

In [ ]:
# @title get Convolutional NN

def get_cnn(num_utterances,
            num_tokens,
            embedding_dim,
            num_emotion_classes,
            output_dim,
            dropout,
            l1,
            l2,
            **kwargs
            ):

  utterance_embeddings_input = Input(shape=(num_utterances, embedding_dim), name="utterance_embeddings_input")
  emotion_vectors_input = Input(shape=(num_utterances, num_emotion_classes),
                                name="emotion_vectors_input")  # Emotion vector for each utteranch

  concatenated_embeddings = Concatenate(axis=2)([utterance_embeddings_input, emotion_vectors_input])


  # First CNN layer is connected to input
  conv_block = Conv1D(64, 5, activation="relu", padding="same")(concatenated_embeddings)
  conv_block = Dropout(dropout)(conv_block)

  conv_block = Conv1D(64, 3, activation="relu", padding="same")(conv_block)
  conv_block = Dropout(dropout)(conv_block)

  # Add dense layers
  flattened = Flatten()(conv_block)
  dense_out = Dense(1024, activation="relu")(flattened)

  cause_prediction = Dense(output_dim, activation="sigmoid")(dense_out)

  # Create the model
  model = Model(inputs=[utterance_embeddings_input, emotion_vectors_input], outputs=cause_prediction)
  return model

In [ ]:
# @title Plot training results

def plot_training_results(model_log: tf.keras.callbacks.History, metrics: list[str] = []):
    loss = model_log.history["loss"]
    val_loss = model_log.history["val_loss"]

    epochs = range(1, len(loss)+1)

    plt.plot(epochs, loss, "g", label="Training loss")
    plt.plot(epochs, val_loss, "r", label="Validation loss")
    plt.title("Training and validation loss")
    plt.legend()
    plt.show()

    for met in metrics:
      plt.figure()
      plt.plot(epochs, model_log.history[met], "g", label=f"Training {met}")
      plt.plot(epochs, model_log.history[f"val_{met}"], "r", label=f"Validation {met}")
      plt.title(f"Training and validation {met}")
      plt.legend()
      plt.show()

# Functions for Training and Hyperparameter Optimization

In [ ]:
# @title Generate emotion predictions

def predict_emotions_conversation(emotion_classifier: Model, conversations):
  emotions_predicted = list()
  # predict emotions conversation by conversation
  for conversation in conversations:
    # determine conversation length
    conv_len = sum([np.any(utt) for utt in conversation])
    # predict emotions of non padding utterances
    emotions_conv = emotion_classifier.predict_on_batch(conversation[:conv_len])
    # add neutral padding for padded utterances
    padding = np.repeat(np.array([[0., 0., 1., 0., 0., 0., 0.]]),
                        data_params["len_conversation"] - conv_len,
                        axis=0)
    emotions_conv = np.append(emotions_conv, padding, 0)
    emotions_predicted.append(emotions_conv)
  return np.array(emotions_predicted)

In [ ]:
# @title Create and train a model

def train_model(x: Sequence, y: Sequence,
                x_valid: Sequence, y_valid: Sequence,
                train_params: dict,
                data_params: dict,
                verbose: int = 1,
                get_model: Callable | None = None
                ) -> Tuple[Model, tf.keras.callbacks.History]:

  # load model
  if get_model is None:
    model = get_fc_nn(num_utterances=data_params["len_conversation"],
                      num_tokens=data_params["len_utterance"],
                      embedding_dim=data_params["len_embedding"],
                      num_emotion_classes=data_params["n_emotions"],
                      output_dim = data_params["len_conversation"]**2,
                      dropout = train_params["dropout"],
                      l1=train_params["l1"],
                      l2=train_params["l2"],)
  else:
    model = get_model(**data_params, **train_params)

  # Compile the model, using adam optimizer and binary crossentropy loss
  adam = optimizers.Adam(learning_rate=train_params["learning_rate"])
  model.compile(optimizer=adam,
                loss="binary_crossentropy",
                metrics=train_params["metrics"])

  # train the model
  hist = model.fit(x=x,
                  y=y,
                  validation_data=(x_valid, y_valid),
                  batch_size=train_params["batch_size"],
                  epochs=train_params["epochs"],
                  callbacks=train_params["callbacks"],
                  verbose=verbose)

  if verbose == 1:
    plot_training_results(hist, ["precision", "recall", "f1_score"])

  return model, hist

In [ ]:
# @title Perform gridsearch over hyperparameters
# TODO: add metrics and callbacks to call structure, remove static references

def grid_search(x: Sequence, y: Sequence,
                x_valid: Sequence, y_valid: Sequence,
                lr: Iterable, lr_freq: Iterable, lr_decay: Iterable,
                do: Iterable, l1: Iterable, l2: Iterable,
                batch_size: Iterable, epochs: Iterable, rep: int):

  params = itertools.product(lr, lr_freq, lr_decay, do, l1, l2, batch_size, epochs)
  keys = ["learning_rate", "lr_freq", "lr_decay", "dropout", "l1", "l2",
          "batch_size", "epochs"]
  list_of_train_params = [dict(zip(keys, values)) for values in params]
  results = list()

  for train_params in list_of_train_params:
    cum_n = 0
    cum_loss = 0
    cum_prec = 0
    cum_reca = 0
    cum_f1 = 0
    start_time = datetime.now()

    for r in range(rep):
      ind_start_time = datetime.now()
      _, hist = train_model(x, y, x_valid, y_valid,
                            train_params, data_params, verbose=0)
      cum_n += hist.epoch[-1] + 1
      cum_loss += np.mean(hist.history["val_loss"][-5])
      cum_prec += np.mean(hist.history["val_precision"][-5])
      cum_reca += np.mean(hist.history["val_recall"][-5])
      cum_f1 += np.mean(hist.history["val_f1_score"][-5])
      print(f"Model {r}: {(datetime.now() - ind_start_time).total_seconds():.6f} " +
            "seconds until convergence")

    print(f"Total time for sample: {(datetime.now() - start_time).total_seconds():.6f} seconds \n")
    results.append({"n_episodes": cum_n / rep,
                    "val_loss": cum_loss / rep,
                    "val_precision": cum_prec / rep,
                    "val_recall": cum_reca / rep,
                    "val_f1_score": cum_f1 / rep,
                    } | train_params)
    K.clear_session()

  return pd.DataFrame(results)

# Run experiments

In [ ]:
# @title Load and preprocess data

TRAIN_DATA = 1030
VALID_DATA = 172

conv, emo, caus, ind = load_batch_conversation(conv_df, ec_pairs_df, EMBEDDINGS_PATH,
               batch_size = TRAIN_DATA)

conv_valid, emo_valid, caus_valid, ind_valid = load_batch_conversation(conv_df, ec_pairs_df, EMBEDDINGS_PATH,
               batch_size = VALID_DATA, validation=True)

print(conv.shape)

# generate utterance embeddings by averaging
utterance = np.mean(conv, axis=2)
utterance_valid = np.mean(conv_valid, axis=2)

utterance_max = np.max(conv, axis=2)
utterance_valid_max = np.max(conv_valid, axis=2)

utterance_cls = conv[:,:,0]
utterance_valid_cls = conv_valid[:,:,0]

# generate predicted emotions
emotion_classifier = tf.keras.models.load_model('/content/drive/MyDrive/bachelor_thesis/models/emo_model_mlp.keras')
emo_pred = predict_emotions_conversation(emotion_classifier, conv)
emo_valid_pred = predict_emotions_conversation(emotion_classifier, conv_valid)

  0%|          | 0/1030 [00:00<?, ?it/s]

  0%|          | 0/172 [00:00<?, ?it/s]

(1030, 24, 40, 1024)


In [ ]:
# @title Load Metrics and Callbacks

# metrics
precision_metric = tf.metrics.Precision(name="precision")
recall_metric = tf.metrics.Recall(name="recall")
f1_score_metric = tf.metrics.F1Score(average="micro", name="f1_score")

# callbacks
def early_stopping_callback():
  return tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",   # Monitor validation loss
    patience=10,          # Number of epochs with no improvement after which training will be stopped
    verbose=1,            # Verbosity mode (1: update messages)
    )

def lr_schedule_callback(frequency: int = 100, factor: float = 0.8):
  def scheduler(epoch, lr):
    if epoch != 0 and epoch % frequency == 0:
      return lr * factor
    else:
      return lr
  return tf.keras.callbacks.LearningRateScheduler(scheduler)

In [ ]:
train_params = {"learning_rate": 5e-5, "dropout": 0.2, "l1": 1e-5, "l2": 1e-4,
                "batch_size": 16, "epochs": 500,
                "metrics": [precision_metric, recall_metric, f1_score_metric],
                "callbacks": [early_stopping_callback(),
                              lr_schedule_callback(frequency=100, factor=0.9)]
                }
for trial in tqdm(range(10)):
  # experiment 1, train model on true emotion vectors
  model_1, hist_1 = train_model((utterance, emo), caus,
                                (utterance_valid, emo_valid), caus_valid,
                                train_params, data_params, 0)
  print(f"Model {trial * 3 + 1}:" +
        f" F1 score: {hist_1.history['val_f1_score'][-1]:.4f}" +
        f" Precision score: {hist_1.history['val_precision'][-1]:.4f}" +
        f" Recall score: {hist_1.history['val_recall'][-1]:.4f}")
  # experiment 2, train model on predicted emotion vectors
  model_2, hist_2 = train_model((utterance, emo_pred), caus,
                                (utterance_valid, emo_valid_pred), caus_valid,
                                train_params, data_params, 0)
  print(f"Model {trial * 3 + 2}:" +
        f" F1 score: {hist_2.history['val_f1_score'][-1]:.4f}" +
        f" Precision score: {hist_2.history['val_precision'][-1]:.4f}" +
        f" Recall score: {hist_2.history['val_recall'][-1]:.4f}")
  # experiment 3, train model on true and predicted emotion vectors
  # create a mixed emotion data set with both true and predicted emo vectors
  exp_3_split = 0.5
  indicies = np.arange(TRAIN_DATA)
  rng.shuffle(indicies)
  ind_emo_pred = indicies[:int(TRAIN_DATA*exp_3_split)]
  emo_mixed = emo.copy()
  emo_mixed[ind_emo_pred] = emo_pred[ind_emo_pred]
  indicies = np.arange(VALID_DATA)
  rng.shuffle(indicies)
  ind_emo_valid_pred = indicies[:int(VALID_DATA*exp_3_split)]
  emo_valid_mixed = emo_valid.copy()
  emo_valid_mixed[ind_emo_valid_pred] = emo_valid_pred[ind_emo_valid_pred]
  model_3, hist_3 = train_model((utterance, emo_mixed), caus,
                                (utterance_valid, emo_valid_mixed), caus_valid,
                                train_params, data_params, 0)
  print(f"Model {trial * 3 + 3}:" +
        f" F1 score: {hist_3.history['val_f1_score'][-1]:.4f}" +
        f" Precision score: {hist_3.history['val_precision'][-1]:.4f}" +
        f" Recall score: {hist_3.history['val_recall'][-1]:.4f}")

  model_1.save(HOME_DIR + f"models/cause_model_big_true_labels_{trial}.keras")
  model_2.save(HOME_DIR + f"models/cause_model_big_pred_labels_{trial}.keras")
  model_3.save(HOME_DIR + f"models/cause_model_big_mix_labels_{trial}.keras")


  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 165: early stopping
Model 1: F1 score: 0.2121 Precision score: 0.6645 Recall score: 0.5644
Epoch 171: early stopping
Model 2: F1 score: 0.1619 Precision score: 0.5878 Recall score: 0.2803
Epoch 157: early stopping
Model 3: F1 score: 0.2121 Precision score: 0.6645 Recall score: 0.5644
Epoch 181: early stopping
Model 4: F1 score: 0.2089 Precision score: 0.6636 Recall score: 0.5400
Epoch 197: early stopping
Model 5: F1 score: 0.1700 Precision score: 0.5797 Recall score: 0.2531
Epoch 168: early stopping
Model 6: F1 score: 0.2089 Precision score: 0.6636 Recall score: 0.5400
Epoch 201: early stopping
Model 7: F1 score: 0.2040 Precision score: 0.6331 Recall score: 0.5033
Epoch 198: early stopping
Model 8: F1 score: 0.1619 Precision score: 0.5739 Recall score: 0.2813
Epoch 159: early stopping
Model 9: F1 score: 0.2040 Precision score: 0.6331 Recall score: 0.5033
Epoch 162: early stopping
Model 10: F1 score: 0.2089 Precision score: 0.6529 Recall score: 0.5503
Epoch 181: early stopping
Mod

In [ ]:
# @title Random search

def random_search(x: Sequence, y: Sequence,
                  x_valid: Sequence, y_valid: Sequence,
                  samples: int):
  for i in tqdm(range(samples)):
    # learning rate is drawn from an exponential distribution
    # roughly 59% in [1e-5, 1e-4] and 94% in [1e-5, 3e-4]
    learning_rate = 1e-5 + rng.exponential(1) * 1e-4
    lr_freq = 100
    lr_decay = 0.9
    dropout = rng.uniform(0, 0.4)
    # l1 and l2 are drawn from an exponential distribution
    # roughly 60% in [0, 1e-4] and 95% in [0, 3e-4]
    l1 = rng.exponential(1)*1e-4
    l2 = rng.exponential(1)*1e-4
    # batch size is drawn from exponential binomial distribution
    # powers of 2 in [4, 128], centered around 16 and 32
    batch_size = 2**(rng.binomial(5, 0.5)+2)
    epochs = 500

    print(f"generate sample {i}")
    print(f"lr: {learning_rate:.2e}, do: {dropout:.4f}, l1: {l1:.2e}," +
          f" l2: {l2:.2e}, batch_size: {batch_size}")
    new_samples = grid_search(x, y, x_valid, y_valid,
                              (learning_rate,), (lr_freq,), (lr_decay,),
                              (dropout,), (l1,), (l2,), (batch_size,), (epochs,),
                              rep=3)
    K.clear_session()

    print(f"save sample {i}\n")
    samples = pd.DataFrame({
      "learning_rate": [], "lr_freq": [], "lr_decay": [], "dropout": [],
      "l1": [], "l2": [], "batch_size": [], "n_episodes": [], "val_loss": [],
      "val_precision": [], "val_recall": [], "val_f1_score": []
    })
    try:
      samples = pd.read_csv("/content/drive/MyDrive/bachelor_thesis/hyperdata_params_samples.csv")
    except OSError:
      print("Could not find previous samples, generate new file")

    merged_samples = pd.concat([samples, new_samples], ignore_index=True)
    merged_samples.to_csv(path_or_buf=f"/content/drive/MyDrive/bachelor_thesis/hyperdata_params_samples.csv",
                          columns=["learning_rate", "lr_freq", "lr_decay",
                                  "dropout", "l1", "l2", "batch_size",
                                  "n_episodes", "val_loss", "val_precision",
                                  "val_recall", "val_f1_score"])

  0%|          | 0/50 [00:00<?, ?it/s]

generate sample 0
lr: 7.07e-05, do: 0.1156, l1: 7.10e-05, l2: 9.71e-05, batch_size: 8


TypeError: train_model() missing 4 required positional arguments: 'x_valid', 'y_valid', 'train_params', and 'data_params'